##### Objective

Notebook covers the implementation and notes linked with the Queue and Stack learnt from https://realpython.com/queue-in-python/

Most of the implementation might be implemented in multiple cells, and in some cases the code would get written as a file to the drive but the code will be here for review. 

### Unlike the other implementations, this notebook will contain the usage of existing python module like Collections, Itertools to implement/ understand the logic

That means, there will be no seperate Class Type for Node that are pushed into Queue or Stack. 

In [1]:
### Implementing custom logger for practice
import logging

quelog = logging.getLogger('quelog')
quelog.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
handler.setLevel(logging.INFO)
logf = logging.Formatter(fmt='%(message)s | %(asctime)s | %(name)s', datefmt='%d-%b | %H-%M')
handler.setFormatter(logf)
quelog.addHandler(handler)

# testing the quelog
quelog.info("Pushing the test message out.")

Pushing the test message out. | 21-Nov | 14-27 | quelog


In [40]:
from collections import deque

class Queue:
    def __init__(self):
        # observe the private element usage
        self._elements = deque()

    def enqueue(self, elem):
        # only used for appending the elements, no return
        self._elements.append(elem)

    def dequeue(self):
        # only returns the element from the left of the queue to 
        # simulate the fifo config
        return self._elements.popleft()

In [6]:
# checking the above implementation

fifo = Queue()

fifo.enqueue('1st')
fifo.enqueue('2st')
fifo.enqueue('3rd')

In [7]:
quelog.warning(fifo.dequeue())  # 1st
quelog.warning(fifo.dequeue())  # 2nd
quelog.warning(fifo.dequeue())  # 3rd
quelog.warning(fifo.dequeue())  # must throw indexerror 

1st | 21-Nov | 06-21 | quelog
2st | 21-Nov | 06-21 | quelog
3rd | 21-Nov | 06-21 | quelog


IndexError: pop from an empty deque

In [12]:
# implementing iter and len methods, with optional initial elements

from collections import deque

class Queue01:
    def __init__(self, *elems) -> None:
        self._elements = deque(elems)

    def __len__(self):
        return len(self._elements)

    def __iter__(self):
        # note, after the __len__ elem is implemented, the len(self) will return length
        while len(self) > 0:
            yield self.dequeue() # note this dequeue method is implemented below for self

    def enqueue(self, elem):
        self._elements.append(elem)

    def dequeue(self):
        return self._elements.popleft()

In [18]:
newFifo = Queue01('5th', '7th', '15th')

In [19]:
for elem in newFifo:
    quelog.info(elem)

5th | 21-Nov | 06-34 | quelog
INFO:quelog:5th
7th | 21-Nov | 06-34 | quelog
INFO:quelog:7th
15th | 21-Nov | 06-34 | quelog
INFO:quelog:15th


In [20]:
# Building the stack is done through inheritance 

class Stack(Queue01):
    def dequeue(self):
        # elements are instantiated using the super class _elements
        return self._elements.pop()

In [21]:
lifo = Stack("6th", '8th', '10th')

for ele in lifo:
    quelog.info(ele)

10th | 21-Nov | 06-39 | quelog
INFO:quelog:10th
8th | 21-Nov | 06-39 | quelog
INFO:quelog:8th
6th | 21-Nov | 06-39 | quelog
INFO:quelog:6th


### Above inheritance is done only for the code reuse. Stack is not a subtype of Queue.  In the real world, you should probably make both classes inherit from an abstract base class because they share the same interface.

Implementing Priority Queue:

In a priority queue, it’s an element’s priority and the insertion order that together determine the ultimate position within the queue.
So the above Queue or Queue01 cannot be reused

An unordered list of elements and their priorities, which you search through every time before dequeuing the element with the highest priority

An ordered list of elements and their priorities, which you sort every time you enqueue a new element

An ordered list of elements and their priorities, which you keep sorted by finding the right spot for a new element using binary search

A binary tree that maintains the heap invariant after the enqueue and dequeue operations

Based on the time complexity involved for sorting with different data structure, the heap datastructure is used

Python has the heapq module, which conveniently provides a few functions that can turn a regular list into a heap and manipulate it efficiently. The two functions that’ll help you build a priority queue are:

heapq.heappush()

heapq.heappop()

- Heap isn’t so much about sorting elements but rather keeping them in a certain relationship to allow for "quick lookup". 

- first element on a heap always has the smallest (min-heap) or the highest (max-heap) value. 

In [25]:
from heapq import heappush

fruits = []

heappush(fruits, 'orange')
heappush(fruits, 'apple')
heappush(fruits, 'banana')

fruits


['apple', 'orange', 'banana']

In [26]:
from heapq import heappop

quelog.info(heappop(fruits))

apple | 21-Nov | 07-27 | quelog
INFO:quelog:apple


In [28]:
quelog.info(fruits)  # first element continues to be smaller

# character with a lower Unicode code point is considered smaller in Python

['banana', 'orange'] | 21-Nov | 07-28 | quelog
INFO:quelog:['banana', 'orange']


In [30]:
person_1 = ('Jane', 'doe', 57)
person_2 = ('Jane', 'smith', 38)
person_3 = ('Noah', 'Smith', 27)

person_1 < person_2  # How python decides which tuple to go first? 

# Once the order is decided (last name in this case), rest of the elements don't matter

# You can enforce a prioritized order on the heap by storing tuples whose first element is a priority. 

True

In [31]:
# implementing priority queue with deque and heapq

class PriorityQueue:
    def __init__(self):
        self._elements = []

    def enqueue_with_priority(self, priority, value):
        heappush(self._elements, (priority, value))

    def dequeue(self):
        return heappop(self._elements)

In [35]:
# priority items

CRITICAL = 3
IMPORTANT = 2
NEUTRAL = 1

# priority values can be ordered in the way implementation wants. If the order changes a new class can be implemented

In [33]:
messages = PriorityQueue()

messages.enqueue_with_priority(IMPORTANT, 'windshield wipers')
messages.enqueue_with_priority(NEUTRAL, 'Radio station tuning')
messages.enqueue_with_priority(CRITICAL, 'Break Pedal depressed')
messages.enqueue_with_priority(CRITICAL, 'Airbag infate')

In [36]:
messages.dequeue()  # observe the neutral element is coming out first... which can be undesirable when priority values are higher

(2, 'windshield wipers')

In [38]:
# Implementing another variation of the class 

class PriorityQueue:
    def __init__(self) -> None:
        self._elements = []

    def enqueue_with_priority(self, priority, value):
        # making the priority negative changes how it is placed in the heapq
        heappush(self._elements, (-priority, value))

    def dequeue(self):
        return heappop(self._elements)[1] # okay, that tripped me. We are returning a tuple remember

In [39]:
messages = PriorityQueue()

messages.enqueue_with_priority(IMPORTANT, 'windshield wipers')
messages.enqueue_with_priority(NEUTRAL, 'Radio station tuning')
messages.enqueue_with_priority(CRITICAL, 'Break Pedal depressed')
messages.enqueue_with_priority(CRITICAL, 'Airbag infate')

In [40]:
messages.dequeue()

'Airbag infate'

In [41]:
messages.dequeue()

'Break Pedal depressed'

Even though the priority queue above keeps the order correctly, but the sorting is unstable. When messages have the same priority the queue should sort them by their insertion order.
This inconsistency happens due to the lexicographic order of comparison between elements in python. So the word Hazard comes before Windshield, which creates the challenge

In [1]:
from dataclasses import dataclass

@dataclass
class Message:
    event: str

wipers = Message("Windshield On")
right = Message("Right light On")

wipers < right

TypeError: '<' not supported between instances of 'Message' and 'Message'

Message objects might be more convenient to work with than plain strings, but unlike strings, they aren’t comparable unless you tell Python how to perform the comparison.

To avoid the comparison of strings, the alternate is to use timestamp. A monotonically increasing counter will do the trick. In other words, you want to count the number of enqueue operations performed without considering the potential dequeue operations that might be taking plac

In [2]:
from collections import deque
from heapq import heappop, heappush
from itertools import count

class PriorityQueue:
    def __init__(self) -> None:
        self._elements = []
        self._counter = count()

    def enqueue_with_priority(self, priority, value):
        elem = (-priority, next(self._counter), value)
        heappush(self._elements, elem)

    def dequeue(self):
        return heappop(self._elements)[-1]

To reuse the code in Python, you find the least common denominator and then extract that code into a MixinClass

In [4]:
class IterableMixin:
    def __len__(self):
        return len(self._elements)

    def __iter__(self):
        # note, after the __len__ elem is implemented, the len(self) will return length
        while len(self) > 0:
            yield self.dequeue() # note this dequeue method is implemented below for self

# To emphasize this difference, some people call it the inclusion of a mixin class rather than pure inheritance.


In [5]:
from collections import deque
from heapq import heappop, heappush
from itertools import count
# Mixins are great for encapsulating behavior rather than state, much like default methods in Java interfaces.
class PriorityQueue(IterableMixin):
    def __init__(self) -> None:
        self._elements = []
        self._counter = count()

    def enqueue_with_priority(self, priority, value):
        elem = (-priority, next(self._counter), value)
        heappush(self._elements, elem)

    def dequeue(self):
        return heappop(self._elements)[-1]

python -m pip install networkx pygraphviz

In [2]:
import networkx as nx

quelog.info(nx.nx_agraph.read_dot('materials-queue/src/roadmap.dot'))

MultiGraph named 'Cities in the United Kingdom' with 70 nodes and 137 edges | 21-Nov | 14-29 | quelog


In [4]:
graph = nx.nx_agraph.read_dot("materials-queue/src/roadmap.dot")
graph.nodes["london"]

{'country': 'England',
 'latitude': '51.507222',
 'longitude': '-0.1275',
 'pos': '80,21!',
 'xlabel': 'City of London',
 'year': '0'}

In [6]:
# Creating a NamedTuple as it is hashable out of the box

from typing import NamedTuple

class City(NamedTuple):
    name: str
    country: str
    year: int | None
    latitude: float
    longitude: float

    @classmethod
    def from_dict(cls, attrs):
        return cls(
            name=attrs['xlabel'],
            country=attrs['country'],
            year=int(attrs['year']),
            latitude=float(attrs['latitude']),
            longitude=float(attrs['longitude'])
            )

In [7]:
# new graph instance and take note of the mapping of node identifiers to city instances.

def load_graph(filename, node_factory: City):
    # 
    graph = nx.nx_agraph.read_dot(filename)
    nodes = {
        name: node_factory(attributes) for name, attributes in graph.nodes(data=True)
    }
    return nodes, nx.Graph(
        (nodes[name1], nodes[name2], weights) for name1, name2, weights in graph.edges(data=True)
    )

In [28]:
node, graph = load_graph('materials-queue/src/roadmap.dot', City.from_dict)

In [21]:
node['london']

City(name='City of London', country='England', year=0, latitude=51.507222, longitude=-0.1275)

In [22]:
print(graph)

Graph with 70 nodes and 137 edges


In [11]:
for neighbor in graph.neighbors(node['london']):
    quelog.info(neighbor.name)

Bath | 21-Nov | 14-44 | quelog
Brighton & Hove | 21-Nov | 14-44 | quelog
Bristol | 21-Nov | 14-44 | quelog
Cambridge | 21-Nov | 14-44 | quelog
Canterbury | 21-Nov | 14-44 | quelog
Chelmsford | 21-Nov | 14-44 | quelog
Coventry | 21-Nov | 14-44 | quelog
Oxford | 21-Nov | 14-44 | quelog
Peterborough | 21-Nov | 14-44 | quelog
Portsmouth | 21-Nov | 14-44 | quelog
Southampton | 21-Nov | 14-44 | quelog
Southend-on-Sea | 21-Nov | 14-44 | quelog
St Albans | 21-Nov | 14-44 | quelog
Westminster | 21-Nov | 14-44 | quelog
Winchester | 21-Nov | 14-44 | quelog


In [24]:
node['edinburgh']

City(name='Edinburgh', country='Scotland', year=1329, latitude=55.953333, longitude=-3.189167)

In [13]:
for neighbor, weights in graph[node["london"]].items():
    quelog.info(f"{weights['distance'], neighbor.name}")

('115', 'Bath') | 21-Nov | 14-48 | quelog
('53', 'Brighton & Hove') | 21-Nov | 14-48 | quelog
('118', 'Bristol') | 21-Nov | 14-48 | quelog
('61', 'Cambridge') | 21-Nov | 14-48 | quelog
('62', 'Canterbury') | 21-Nov | 14-48 | quelog
('40', 'Chelmsford') | 21-Nov | 14-48 | quelog
('100', 'Coventry') | 21-Nov | 14-48 | quelog
('58', 'Oxford') | 21-Nov | 14-48 | quelog
('85', 'Peterborough') | 21-Nov | 14-48 | quelog
('75', 'Portsmouth') | 21-Nov | 14-48 | quelog
('79', 'Southampton') | 21-Nov | 14-48 | quelog
('42', 'Southend-on-Sea') | 21-Nov | 14-48 | quelog
('25', 'St Albans') | 21-Nov | 14-48 | quelog
('1', 'Westminster') | 21-Nov | 14-48 | quelog
('68', 'Winchester') | 21-Nov | 14-48 | quelog


In [14]:
def sort_by(neighbor, strategy):
    return sorted(neighbor.items(), key = lambda item: strategy(item[1]))

def by_distance(weights):
    return float(weights["distance"])

for neighbor, weights in sort_by(graph[node['london']], by_distance):
    quelog.info(f"{weights['distance']:>3} miles, {neighbor.name}")

  1 miles, Westminster | 21-Nov | 15-20 | quelog
 25 miles, St Albans | 21-Nov | 15-20 | quelog
 40 miles, Chelmsford | 21-Nov | 15-20 | quelog
 42 miles, Southend-on-Sea | 21-Nov | 15-20 | quelog
 53 miles, Brighton & Hove | 21-Nov | 15-20 | quelog
 58 miles, Oxford | 21-Nov | 15-20 | quelog
 61 miles, Cambridge | 21-Nov | 15-20 | quelog
 62 miles, Canterbury | 21-Nov | 15-20 | quelog
 68 miles, Winchester | 21-Nov | 15-20 | quelog
 75 miles, Portsmouth | 21-Nov | 15-20 | quelog
 79 miles, Southampton | 21-Nov | 15-20 | quelog
 85 miles, Peterborough | 21-Nov | 15-20 | quelog
100 miles, Coventry | 21-Nov | 15-20 | quelog
115 miles, Bath | 21-Nov | 15-20 | quelog
118 miles, Bristol | 21-Nov | 15-20 | quelog


### Implementing the BFS using the Graph and Queue

- BFS looks for a node that satisfies a condition, by exploring the graph in concentric layers / levels

- Source node is **arbitrarily** chosen, unless the graph is a tree in which case traversal happens from root node

In [25]:
nx.bfs_tree(G=graph, source=node['edinburgh'])

In [32]:
def is_twentieth_century(year):
    return year and 1901 <= year <= 2000

for item in nx.bfs_tree(graph, node['edinburgh']):
    # quelog.info(f"{item.name}")
    if is_twentieth_century(item.year):
        quelog.info(f"Found: {item.name, item.year}")

Found: ('Lancaster', 1937) | 21-Nov | 15-36 | quelog
Found: ('Sunderland', 1992) | 21-Nov | 15-36 | quelog
Found: ('Salford', 1926) | 21-Nov | 15-36 | quelog
Found: ('Stoke-on-Trent', 1925) | 21-Nov | 15-36 | quelog
Found: ('St Davids', 1994) | 21-Nov | 15-36 | quelog
Found: ('Swansea', 1969) | 21-Nov | 15-36 | quelog
Found: ('Derby', 1977) | 21-Nov | 15-36 | quelog
Found: ('Cardiff', 1905) | 21-Nov | 15-36 | quelog
Found: ('Leicester', 1919) | 21-Nov | 15-36 | quelog
Found: ('Cambridge', 1951) | 21-Nov | 15-36 | quelog
Found: ('Plymouth', 1928) | 21-Nov | 15-36 | quelog
Found: ('Southampton', 1964) | 21-Nov | 15-36 | quelog
Found: ('Portsmouth', 1926) | 21-Nov | 15-36 | quelog


In [37]:
def breadth_first_traverse(graph, source):
    # create a queue object
    queue = Queue(source)
    # create a set for visited nodes
    visited = {source}
    while queue:
        # dequeue an item from the queue, assign it to the variable node, and simultaneously yield that value.
        yield (node := queue.dequeue())
        # get the neighbor of the node
        for neighbor in graph.neighbors(node):
            # check if neighbor is in visited
            if neighbor not in visited:
                # add it to the visited
                visited.add(neighbor)
                # enqueue it to the queue
                queue.enqueue(neighbor)

In [38]:
def breadth_first_search(graph, source, predicate):
    for node in breadth_first_traverse(graph, source):
        if predicate(node):
            return node

In [41]:
city = breadth_first_search(graph, node['edinburgh'], is_twentieth_century)

TypeError: Queue.__init__() takes 1 positional argument but 2 were given